# {state} Highways with No Parallel Transit Routes to SHN 

Based off of `C7`. 

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import branca
import calitp.magics
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown, HTML

import setup_corridors_stats
from create_parallel_corridors import IMG_PATH, DATA_PATH
from shared_utils import geography_utils, map_utils
from shared_utils import calitp_color_palette as cp

catalog = intake.open_catalog("./*.yml")

In [ ]:
operator_stats, hwy_stats = setup_corridors_stats.aggregated_transit_hwy_stats()
gdf = setup_corridors_stats.process_hwy_stats(hwy_stats).to_crs(geography_utils.WGS84)

## Highways with Zero Parallel Routes

These indicate a dearth of bus transit service.

Sort in descending order where there are more bus routes, just that these aren't parallel bus routes, and then sort in decreasing length of highway corridor.

Show top 25 and make district maps. (top 20 includes 9 districts, top 25 includes 11 districts, top 30 includes 11 districts, top 35 includes 12 districts). Stick with top 25 then.

In [ ]:
TOP_CUTOFF = 25
zero_parallel = (gdf[gdf.pct_parallel==0]
                 .sort_values(["count_route_id", "highway_length_routetype"], 
                              ascending = [False, False])
                 .head(TOP_CUTOFF)
                ).reset_index(drop=True)

In [ ]:
def district_stats(gdf, district):
    subset = gdf[gdf.District==district]
        
    display_cols = [
            "Route", "County", "RouteType", 
            "count_route_id", "highway_length_routetype",
        ]

    # Format html table
    subset_style = (
        subset[display_cols]
        .rename(columns = POPUP_DICT)
        .style.format({'Hwy Length (mi)': '{:,.2f}'})
        .set_properties(**{'text-align': 'center'})
        .set_table_styles([dict(selector='th',props=[('text-align', 'center')])
                          ])
        .hide(axis="index")
        .to_html()
    )
    if len(subset) > 0:
        display(HTML(subset_style))

In [ ]:
FIG_HEIGHT = 320
FIG_WIDTH = 300

PLOT_COL = "count_route_id"
POPUP_DICT = {
    "Route": "Hwy Route",
    "County": "County",
    "District": "District",
    "RouteType": "Route Type",
    "count_route_id": "# transit routes",
    "highway_length_routetype": "Hwy Length (mi)",
}

def make_map(gdf): 
    # Create unique colors for each highway in district
    # Do it off of the index value
    index = gdf.index
    COLORSCALE = branca.colormap.StepColormap(
        colors = (cp.CALITP_CATEGORY_BRIGHT_COLORS + cp.CALITP_CATEGORY_BOLD_COLORS)[index],
    )
    
    m = map_utils.make_folium_choropleth_map(
        gdf,
        plot_col = PLOT_COL,
        popup_dict = POPUP_DICT, tooltip_dict = POPUP_DICT,
        colorscale = COLORSCALE,
        fig_width = FIG_WIDTH,
        fig_height = FIG_HEIGHT,
        zoom = 10,
        centroid = [gdf.geometry.centroid.y, 
                    gdf.geometry.centroid.x,
                   ],
        title = f"{gdf.hwy_route_name.iloc[0]}",
        legend_name = "",
    )
    
    return m

In [ ]:
def show_district_analysis(gdf, district):
    subset = (gdf[gdf.District==district]
              .sort_values(
                  ["count_route_id", "highway_length_routetype"], 
                  ascending=[False, False])
              .reset_index(drop=True)
             )
    
    # Show a table of stats for entire district
    district_stats(subset, district)
    
    for i in range(0, len(subset)):
        m = make_map(subset[subset.index==i])
        display(m)


In [ ]:
plot_df = zero_parallel.copy()

def sentence_no_corridors(district):
    sentence = (
        f"District {district} does not contain any of the "
        f"top {TOP_CUTOFF} highway corridors with no "
        "parallel bus routes to the SHN."
    )
    return sentence

def district_output(gdf, district):
    try:
        show_district_analysis(gdf, district)
    except:
        print(sentence_no_corridors(district))

## District Maps
### District 1

In [ ]:
district = 1
district_output(plot_df, district)

### District 2

In [ ]:
district = 2
district_output(plot_df, district)

### District 3

In [ ]:
district = 3
district_output(plot_df, district)

### District 4

In [ ]:
district = 4
district_output(plot_df, district)

### District 5

In [ ]:
district = 5
district_output(plot_df, district)

### District 6

In [ ]:
district = 6
district_output(plot_df, district)

### District 7

In [ ]:
district = 7
district_output(plot_df, district)

### District 8

In [ ]:
district = 8
district_output(plot_df, district)

### District 9

In [ ]:
district = 9
district_output(plot_df, district)

### District 10

In [ ]:
district = 10
district_output(plot_df, district)

### District 11

In [ ]:
district = 11
district_output(plot_df, district)

### District 12

In [ ]:
district = 12
district_output(plot_df, district)